In [11]:
import pybert as pb
import pygimli as pg
import numpy as np
import pygimli.meshtools as mt
from pygimli.viewer.pv import drawSensors
from pygimli.physics.ert import simulate as simulateERT
from pygimli.viewer.pv import drawStreamLines, drawSlice

In [12]:
inputfileERT = '3d_data_test.data'
elecs_raw = np.genfromtxt('elecsXYZ.csv', delimiter=",",skip_header=1)
dataERT = pb.load(inputfileERT) # pick up the scheme
dataHom = pb.load('synthetic/homogeneous.ohm') # pick up the k factor
mesh_inv = pg.load('synthetic/mesh_inv.bms') # pick up the mesh
dataERT.set('k',(dataHom('u') / dataHom('i')))
dataERT.set('rhoa', dataERT('k') * dataERT('r'))

05/03/21 - 12:58:13 - pyGIMLi - INFO - imported:  Data: Electrodes: 48 data: 2194
05/03/21 - 12:58:13 - pyGIMLi - INFO - imported:  Data: Electrodes: 48 data: 2194


In [ ]:
ert = pg.physics.ERTManager()  # sr=False, verbose=True, debug=False)
dataERT['err'] = ert.estimateError(dataERT, 
                                   absoluteError=0.001, 
                                   relativeError=0.1)
dataERT.markInvalid(dataERT("rhoa") < 0)
dataERT.removeInvalid()
dataERT.save('dataERT_f.data')
model = ert.invert(data=dataERT, mesh=mesh_inv,lam=20,verbose=True)
pg.info('Inversion stopped with chi² = {0:.3}'.format(ert.fw.chi2()))

05/03/21 - 12:58:22 - pyGIMLi - INFO - Found 1 regions.
05/03/21 - 12:58:22 - pyGIMLi - INFO - Creating forward mesh from region infos.


In [ ]:
plotter, _ = pg.show(mesh_inv, alpha=0.1, data=model, hold=True,notebook=True)
drawSlice(plotter, mesh_inv, normal=[0,1,0], data=model, label="model")
drawSensors(plotter,  dataERT.sensors(), diam=5, color='yellow')
plotter.show()

mesh_inv.addData('model_real_inv',model)
mesh_inv.exportVTK('model_real_inv.vtk')

cov = ert.coverage()
plotter, _ = pg.show(mesh_inv, data=cov,
                     alpha=0.9, hold=True, 
                     label="Coverage", 
                     notebook=True)
plotter.show()